# Import Libraries

In [1]:
from read_data import combine_all, combine_category, read_data

In [15]:
import pandas as pd    
import numpy as np      
import os
from stop_words import get_stop_words

In [5]:
import nltk 
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [16]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

# Scrapped Data

In [6]:
df=combine_all()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1101345 entries, 0 to 60532
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Name         1101261 non-null  object
 1   Comment      1101078 non-null  object
 2   Time         1101345 non-null  object
 3   Likes        1101345 non-null  int64 
 4   Reply Count  1101345 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 50.4+ MB


# Cleaning Comments from the data

In [17]:
df["Comment"]=df["Comment"].astype(str)

In [9]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# Download NLTK Modules

In [10]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srijanmalhotra/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/srijanmalhotra/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/srijanmalhotra/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/srijanmalhotra/nltk_data...


True

In [13]:
df["Comment"] = df["Comment"].apply(lambda x: clean_text(x))

In [14]:
sentid = SentimentIntensityAnalyzer()
df["sentiments"] = df["Comment"].apply(lambda x:sentid.polarity_scores(x))
df = pd.concat([df.drop(["sentiments"],axis=1),df["sentiments"].apply(pd.Series)],axis=1)
df

,Name,Comment,Time,Likes,Reply Count,neg,neu,pos,compound
0,MrBeast,like say video subscribe haven’t already could...,2021-03-27T23:31:32Z,260829,419,0.000,0.488,0.512,0.7430
1,Alisha Gouker,look scary,2022-04-02T23:34:40Z,0,0,0.762,0.238,0.000,-0.4939
2,Lea Wodi,work breathe mean time air go,2022-04-02T23:20:34Z,0,0,0.000,1.000,0.000,0.0000
3,Alex gamer,oxygen,2022-04-02T21:19:16Z,0,0,0.000,1.000,0.000,0.0000
4,Deedee Brown,oh know could,2022-04-02T20:43:11Z,0,0,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...
60528,Poke ax,wow,2021-04-07T18:45:37Z,11,9,0.000,0.000,1.000,0.5859
60529,Friday game week,first,2021-04-07T18:45:37Z,7,26,0.000,1.000,0.000,0.0000
60530,Zaptix,first,2021-04-07T18:45:36Z,16,14,0.000,1.000,0.000,0.0000
60531,ROTNOX,first,2021-04-07T18:45:36Z,10,11,0.000,1.000,0.000,0.0000
